In [1]:
# crawl MITRE ATT&CK
#  - crawl Technique texts
#  - crawl mitigation code and link

import os
import json
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict
from urllib.request import Request, urlopen

tech_dict = defaultdict(dict)
'''
{
    'code' : {
        'name': str,
        'link': str,
        'desc': str,
        'miti': [str],
        'miti_link': [str]
    }
}
'''

save_dir = '/data/zhaohan/adv-reasoning/data/cyberkg-raw/mitre-attack/'
os.makedirs(save_dir, exist_ok=True)

# NOTE: we only consider enterprise tech
root = 'https://attack.mitre.org/techniques/enterprise/'
response = requests.get(root)
html = BeautifulSoup(response.content, 'html.parser')

link_prefix = 'https://attack.mitre.org'

for tr in tqdm(html.find_all('tr', attrs={'class':'technique'})):
    link = link_prefix + tr.find('a').get('href')
    
    if link.split('/')[-1].startswith('T'):
        code = link.split('/')[-1]
    else:
        code = '.'.join(link.split('/')[-2:])
    
    tech_html = BeautifulSoup(requests.get(link).content, 'html.parser')
    name = ' '.join(tech_html.find('h1').text.strip().replace('\n', ' ').split()) 
    try:
        desc = tech_html.find('div', attrs={'class':'description-body'}).text.strip()
    except: 
        print(code, name, link)
        desc = ''
        
    miti_codes = []
    miti_links = []
    for a in tech_html.find_all('a'):
        url = a.get('href')
        if url is not None and 'mitigations' in url and url.split('/')[-1].startswith('M'):
            m_code = url.split('/')[-1]
            if m_code not in miti_codes:
                miti_codes.append(m_code)
                miti_links.append(link_prefix + url)
#     print(code, name, link, miti_codes, miti_links)
    
    tech_dict[code] = {
        'name': name,
        'link': link,
        'desc': desc,
        'miti': miti_codes,
        'miti_link': miti_links,
    }
    
with open(os.path.join(save_dir, 'techniques.json'), 'w') as f: 
    json.dump(tech_dict, f)
print('Done')

 78%|███████▊  | 440/567 [02:08<00:38,  3.32it/s]

T1053.004 Scheduled Task/Job: Launchd https://attack.mitre.org/techniques/T1053/004


100%|██████████| 567/567 [02:45<00:00,  3.43it/s]


Done
